### Tasks
1. Determine the performance of storing vehicle roster as a pandas dataframe. 
2. Compare the departure time profile with smoothing to the one without


Change log

v1: 2018-12-07

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [53]:
import os
from time import time

In [29]:
DYNUS_T_DATA_DIR = 'C:\Projects\Repo\Work\SWIFT\data\Dynus_T'
vehicle_roster_file_deliv = 'vehicles_deliv_17.csv'
vehicle_roster_file_deliv = os.path.join(DYNUS_T_DATA_DIR, vehicle_roster_file_deliv)
vehicle_roster_file_deliv

'C:\\Projects\\Repo\\Work\\SWIFT\\data\\Dynus_T\\vehicles_deliv_17.csv'

In [30]:
# The file has two header rows. The second row has the column defs. 


# 55669           1    # of vehicles in the file, Max # of STOPs
#  #   usec   dsec   stime vehcls vehtype ioc #ONode #IntDe info ribf    comp   izone Evac InitPos    VoT  tFlag pArrTime TP IniGas
#  1 115449 278035    12.7     3     6     1     1     1     0  0.0000  0.0000     5     0  0.92110000    0.00    0    0.0    5  0.0
#    61   0.00

dtypes = {
    '#':np.int32,
    'usec': np.int32,   
    'dsec': np.int32,   
    'stime': np.float32, 
    'vehcls': np.uint8, 
    'vehtype': np.uint8, 
    'ioc': np.uint8, 
    '#ONode': np.uint32, 
    '#IntDe': np.uint16, 
    'info': np.uint8,
    'ribf': np.float16,    
    'comp': np.float16,   
    'izone': np.uint16, 
    'Evac': np.uint8, 
    'InitPos': np.float32,    
    'VoT': np.float32,  
    'tFlag': np.uint8, 
    'pArrTime': np.float16, 
    'TP': np.uint8, 
    'IniGas': np.float16,
    'DZone': np.uint16,
    'waitTime': np.float
}


In [56]:
start_time = time()
df_vehicles_deliv = pd.read_csv(vehicle_roster_file_deliv, skiprows=0, header=0,
                                sep=',', 
                                dtype=dtypes,  
                               )
end_time = time()
print("{0:d} trips read in {1:.0f} seconds".format(df_vehicles_deliv.shape[0], (end_time-start_time)))

19301935 trips read in 77 seconds


In [32]:
df_vehicles_deliv.head()

,vid,usec,dsec,stime,vehcls,vehtype,ioc,#ONode,#IntDe,info,...,Izone,Evac,InitPos,VoT,tFlag,pArrTime,TP,IniGas,DZone,waitTime
0,1,252555,14547,0.0,3,1,0,8,1,0,...,2774,0,0.788086,23.656250,0,0.0,2,0.0,2782,0.0
1,2,322244,313074,0.0,3,1,0,12,1,0,...,2816,0,0.675781,23.656250,0,0.0,2,0.0,2729,0.0
2,3,271541,82019,0.0,3,6,1,123,1,0,...,961,0,0.284668,64.000000,0,0.0,5,0.0,5248,0.0
3,4,306502,302598,0.0,3,5,1,28,1,0,...,5077,0,0.662109,40.000000,0,0.0,5,0.0,430,0.0
4,5,24808,294441,0.0,3,1,0,2,1,0,...,328,0,0.633789,20.484375,0,0.0,1,0.0,328,0.0


In [94]:
df_vehicles_deliv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19301935 entries, 0 to 19301934
Data columns (total 22 columns):
Izone       int64
DZone       uint64
TP          uint64
ioc         uint64
vehtype     uint64
vid         int64
usec        int32
dsec        int32
stime       float16
vehcls      uint8
#ONode      uint32
#IntDe      uint16
info        uint8
ribf        float16
comp        float16
Evac        uint8
InitPos     float16
VoT         float16
tFlag       uint8
pArrTime    float16
IniGas      float16
waitTime    float64
dtypes: float16(7), float64(1), int32(2), int64(2), uint16(1), uint32(1), uint64(4), uint8(4)
memory usage: 1.6 GB


In [67]:
# stime mean/std probably overflowed.
min_stime, max_stime = df_vehicles_deliv.stime.values.min(), df_vehicles_deliv.stime.values.max()
min_vot, max_vot = df_vehicles_deliv.VoT.values.min(), df_vehicles_deliv.VoT.values.max()
min_zone, max_zone = 1, 5262
min_tp, max_tp = df_vehicles_deliv.TP.values.min(), df_vehicles_deliv.TP.values.max()
min_ioc, max_ioc = df_vehicles_deliv.ioc.values.min(), df_vehicles_deliv.ioc.values.max()
min_vehtype, max_vehtype = df_vehicles_deliv.vehtype.values.min(), df_vehicles_deliv.vehtype.values.max()

In [57]:
index_columns = ['Izone', 'DZone', 'TP', 'ioc', 'vehtype']
df_vehicles_deliv.groupby(by=index_columns)['vid'].count().describe()

,vid,usec,dsec,stime,vehcls,#ONode,#IntDe,info,ribf,comp,Evac,InitPos,VoT,tFlag,pArrTime,IniGas,waitTime
count,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06,6.424061e+06
mean,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00,3.004631e+00
std,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01,1.124951e+01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
50%,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
75%,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00
max,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03,3.056000e+03


Most OD pairs only have only a few trips. 

#### Generate random instructions

In [98]:
df_vehicles_deliv.set_index(index_columns, inplace=True)
df_vehicles_deliv.sort_index(inplace=True)

In [69]:
NUMBER_OF_INSTRUCTIONS = 100

In [84]:
def generate_instructions(n):
    """
    n: number of instructions to generate
    """
    
    global min_stime, max_stime, min_vot, max_vot, min_zone, max_zone, min_tp, max_tp, min_ioc, max_ioc
    global min_vehtype, max_veh_type
    
    stime_lb = np.random.uniform(low=min_stime, high=max_stime, size=n)
    stime_buffer = np.random.randint(4, size=n)
    stime_hb = np.clip(stime_lb + stime_buffer, a_min=0, a_max=max_stime)
    
    vot_lb = np.random.uniform(low=min_vot, high=max_vot, size=n)
    vot_buffer = np.random.randint(15, size=n)
    vot_hb = np.clip(vot_lb + vot_buffer, a_min=min_vot, a_max=max_stime)
    
    origins = np.random.choice(np.arange(1,max_zone+1), replace=True, size=n)
    destinations = np.random.choice(np.arange(1,max_zone+1), replace=True, size=n)
    
    purposes = np.random.choice(np.arange(1,max_tp+1), replace=True, size=n)
    occs = np.random.choice(np.arange(1,max_ioc+1), replace=True, size=n)
    vehtypes = np.random.choice(np.arange(1,max_vehtype+1), replace=True, size=n)
    
    changes = np.random.choice(np.arange(1,11), replace=True, size=n)
    
    return np.column_stack((origins, destinations, purposes, occs, vehtypes, stime_lb, stime_hb, vot_lb, vot_hb))
    

In [87]:
instructions = generate_instructions(5)
instructions.shape

(5, 9)

In [86]:
instructions.shape

(5, 9)

In [88]:
def timeit(func):
    def f(*args, **kwargs):
        args = list(args)
        start_time = time.time()
        rv = func(*args, **kwargs)
        end_time = time.time()
        print("Execution time = {0:.0f} sec".format(end_time-start_time))
        return rv
    return f

In [92]:
@timeit
def find_vehicles(df_vehicles, instructions):
    vehicle_count = 0
    for instruction in instructions:
        anchor = tuple(instruction[:5])
        stime_lb, stime_hb, vot_lb, vot_hb = tuple(instruction[5:])
        selection_time = (df_vehicles.stime >= stime_lb) & (df_vehicles.stime <= stime_hb)
        selection_vot = (df_vehicles.VoT >= vot_lb) & (df_vehicles.VoT <= vot_hb)
        chosen = df_vehicles.loc[selection_time & selection_vot].loc[anchor]
        vehicle_count += chosen.shape[0]
    return vehicle_count

In [91]:
df_vehicles_deliv.loc[(2774, 2782, 2, 0, 1)]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


vid    usec    dsec     stime  vehcls  \
Izone DZone TP ioc vehtype                                               
2774  2782  2  0   1               1  252555   14547     0.000       3   
                   1           80058   14547   75154   113.875       3   
                   1          108423  301548   73132   154.375       3   
                   1          256098  271250  301548   270.500       3   
                   1          262121   78598  230244   273.250       3   
                   1          475460   14547  306207   327.000       3   
                   1          552961   78598  106398   340.500       3   
                   1          565512  294319  271249   342.750       3   
                   1          582693   78598  230244   345.750       3   
                   1          593867  271250  301548   347.500       3   
                   1          746955   14547   75154   366.000       3   
                   1         1600053   14547   75154   424.750       3   
                   1         2853188   18357  278090   471.500       3   
                   1         3862572   18357  278090   530.500       3   
                   1         4119663  301548   73132   548.000       3   
                   1         4301379  206554   14801   560.500       3   
                   1         4374725  301548   73132   565.500       3   
                   1         4718099   14801   75154   589.500       3   
                   1         4835502   14801  239302   597.500       3   
                   1         4852411   14547  306207   598.500       3   
                   1         4871788   14547   75154   600.000       3   
                   1         4990743  301548   73132   607.000       3   
                   1         4998227  301548   73132   607.500       3   
                   1         5012386   18357  278090   608.500       3   
                   1         5023386   14801  239302   609.000       3   
                   1         5237184   78598  106398   622.000       3   
                   1         5356869  206554   14801   629.000       3   
                   1         5587426   78598  106398   643.000       3   
                   1         5925568   14547  306207   662.500       3   
                   1         6038747   18357   24111   668.500       3   
...                              ...     ...     ...       ...     ...   
                   1        17372080  301548   73132  1176.000       3   
                   1        17385546  271250  301548  1177.000       3   
                   1        17427881  294319  271249  1180.000       3   
                   1        17621873   14547   75154  1191.000       3   
                   1        17685112   14547   75154  1195.000       3   
                   1        17860359   14547  306207  1208.000       3   
                   1        17927553  294319  271249  1214.000       3   
                   1        17955467   14547  306207  1217.000       3   
                   1        17974681   14547  306207  1219.000       3   
                   1        18047178  206554   14801  1225.000       3   
                   1        18059401  294319  271249  1226.000       3   
                   1        18070671   14547  306207  1227.000       3   
                   1        18134103   14801   75154  1233.000       3   
                   1        18165806  301548   73132  1236.000       3   
                   1        18168134   14801   75154  1236.000       3   
                   1        18318228   14547   75154  1249.000       3   
                   1        18338393   78598   75154  1251.000       3   
                   1        18512941   14547  306207  1269.000       3   
                   1        18708510   78598  230244  1293.000       3   
                   1        18775960  206554   14801  1301.000       3   
                   1        18860576   14801   75154  1311.000       3   
                   1        18871417 